In [1]:
import torch
from torchvision.transforms import v2
from torch.utils.data import DataLoader, Dataset
import os
from sklearn.svm import OneClassSVM

from img_methods import *

In [2]:
H, W = 200, 200

img_aug = v2.Compose([
    v2.Grayscale(num_output_channels=1),
    v2.ToDtype(torch.float32, scale=True),
    v2.PILToTensor(),
])

class DinoNuggieDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root = root_dir
        self.transform = transform
        self.file_list = []
        
        for folder in os.listdir(root_dir):
            if folder == 'mis':
                continue
            folder_fp = os.path.join(root_dir, folder) 
            for img in os.listdir(folder_fp):
                self.file_list.append(os.path.join(folder_fp, img))
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        img = Image.open(self.file_list[idx])
        if self.transform:
            img = self.transform(img)
        return img
    
    
current_dir = os.getcwd()
project_dir = os.path.dirname(current_dir)
images_dir = os.path.join(project_dir, 'images')
    
normal_dataset = DinoNuggieDataset(root_dir = images_dir, transform=img_aug)
normal_dataloader = DataLoader(normal_dataset, batch_size=1, shuffle=True)


In [3]:
normal_flattened = [torch.flatten(img).numpy() for batch in normal_dataloader for img in batch]

In [4]:
dino_nuggie_svm = OneClassSVM(gamma='auto', kernel='rbf', nu=0.05)
dino_nuggie_svm.fit(normal_flattened)

normal_predict = dino_nuggie_svm.predict(normal_flattened)
np.array(np.unique(normal_predict, return_counts=True)).T

array([[ -1, 152],
       [  1,  54]], dtype=int64)